# Weather Informed Reservoir Operation

ResOpsUSData

- URL: https://zenodo.org/record/6612040
- Window: Daily records from 2006-01-01 to 2018-12-31
- Steyaert, J.C., Condon, L.E., W.D. Turner, S. et al. ResOpsUS, a dataset of historical reservoir operations in the contiguous United States. Sci Data 9, 34 (2022). https://doi.org/10.1038/s41597-022-01134-7

In [ ]:
import warnings
warnings.simplefilter("ignore")

from datetime import datetime, date
import pandas as pd
import numpy as np

import seaborn as sns
sns.set_theme(style="white")
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Loading ResOps data

In [ ]:
# from time_series_single_variable_table

daily_inflow = pd.read_csv('Reservoir_project/ResOpsUS/time_series_single_variable_table/DAILY_AV_INFLOW_CUMECS.csv')
daily_outflow = pd.read_csv('Reservoir_project/ResOpsUS/time_series_single_variable_table/DAILY_AV_OUTFLOW_CUMECS.csv')
daily_storage = pd.read_csv('Reservoir_project/ResOpsUS/time_series_single_variable_table/DAILY_AV_STORAGE_MCM.csv')

In [ ]:
daily_inflow.head()

,date,100,1000,1001,1002,1003,1005,1006,1007,101,...,981,982,983,987,989,99,991,993,995,998
0,1980-01-01,NaN,NaN,1.416,NaN,1.699,NaN,9.911,1.416,NaN,...,NaN,0.142,NaN,NaN,22.653,NaN,NaN,NaN,NaN,NaN
1,1980-01-02,NaN,NaN,1.416,NaN,1.699,NaN,8.495,1.416,NaN,...,NaN,0.142,NaN,NaN,22.653,NaN,NaN,NaN,NaN,NaN
2,1980-01-03,NaN,NaN,1.416,NaN,1.416,NaN,8.495,1.416,NaN,...,NaN,0.142,NaN,NaN,24.069,NaN,NaN,NaN,NaN,NaN
3,1980-01-04,NaN,NaN,1.416,NaN,0.850,NaN,7.079,0.850,NaN,...,NaN,0.142,NaN,NaN,24.069,NaN,NaN,NaN,NaN,NaN
4,1980-01-05,NaN,NaN,1.416,NaN,0.850,NaN,5.663,0.850,NaN,...,NaN,0.142,NaN,NaN,19.822,NaN,NaN,NaN,NaN,NaN


In [ ]:
daily_outflow.head()

,date,100,1000,1001,1002,1003,1005,1006,1007,101,...,981,982,983,987,989,99,991,993,995,998
0,1980-01-01,26.221,13.167,1.416,NaN,4.672,NaN,1.133,4.814,NaN,...,1.076,0.566,2.633,NaN,24.777,6.116,3.596,NaN,NaN,0.010
1,1980-01-02,28.883,8.155,1.416,NaN,4.672,NaN,1.133,4.814,NaN,...,0.765,0.566,2.633,NaN,24.777,7.334,3.596,NaN,NaN,0.010
2,1980-01-03,29.733,2.322,1.416,NaN,4.672,NaN,1.133,4.814,NaN,...,0.680,0.566,2.605,NaN,24.777,7.334,3.596,NaN,NaN,0.010
3,1980-01-04,30.299,2.322,1.416,NaN,2.010,NaN,1.133,3.171,NaN,...,0.680,0.566,2.605,NaN,24.777,7.334,3.596,NaN,NaN,0.122
4,1980-01-05,44.741,2.322,1.416,NaN,0.425,NaN,1.133,1.982,NaN,...,0.878,0.566,2.520,NaN,24.777,14.583,3.596,NaN,NaN,0.048


In [ ]:
daily_storage.head()

,date,100,1000,1001,1002,1003,1005,1006,1007,101,...,981,982,983,987,989,99,991,993,995,998
0,1980-01-01,191.969,299.229,0.363,NaN,27.652,NaN,1047.590,27.043,NaN,...,62.399,168.099,104.520,72.836,1495.347,4.217,88.975,NaN,NaN,19.489
1,1980-01-02,192.356,298.334,0.363,NaN,27.456,NaN,1047.982,26.781,NaN,...,62.475,168.048,104.291,73.422,1494.656,4.267,89.324,NaN,NaN,19.982
2,1980-01-03,192.356,298.222,0.363,NaN,27.162,NaN,1048.495,26.491,NaN,...,62.524,167.996,104.138,74.170,1495.347,4.128,89.441,NaN,NaN,20.352
3,1980-01-04,192.356,298.424,0.363,NaN,27.001,NaN,1048.691,26.288,NaN,...,62.645,167.944,104.061,74.944,1495.347,4.259,88.741,NaN,NaN,20.846
4,1980-01-05,192.938,298.614,0.363,NaN,27.001,NaN,1049.156,26.114,NaN,...,62.964,167.893,103.679,75.630,1493.276,4.489,90.023,NaN,NaN,21.216


In [ ]:
daily_inflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Columns: 679 entries, date to 998
dtypes: float64(678), object(1)
memory usage: 77.7+ MB


In [ ]:
daily_outflow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Columns: 679 entries, date to 998
dtypes: float64(678), object(1)
memory usage: 77.7+ MB


In [ ]:
daily_storage.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Columns: 679 entries, date to 998
dtypes: float64(678), object(1)
memory usage: 77.7+ MB


In [ ]:
# from attributes

time_series_inventory = pd.read_csv('Reservoir_project/ResOpsUS/attributes/time_series_inventory.csv')
time_series_inventory_variables = pd.read_csv('Reservoir_project/ResOpsUS/attributes/time_series_inventory_variables.csv')
time_series_variables = pd.read_csv('Reservoir_project/ResOpsUS/attributes/time_series_variables.csv')

reservoir_attributes = pd.read_csv('Reservoir_project/ResOpsUS/attributes/reservoir_attributes.csv')
reservoir_attributes_variables = pd.read_csv('Reservoir_project/ResOpsUS/attributes/reservoir_attributes_variables.csv')

agency_attributes = pd.read_csv('Reservoir_project/ResOpsUS/attributes/agency_attributes.csv')
agency_attributes_variables = pd.read_csv('Reservoir_project/ResOpsUS/attributes/agency_attributes_variables.csv')

In [ ]:
time_series_inventory.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   DAM_ID             678 non-null    int64 
 1   STORAGE            678 non-null    int64 
 2   STORAGE_START      653 non-null    object
 3   STORAGE_END        653 non-null    object
 4   DATA_SOURCE        653 non-null    object
 5   INFLOW             678 non-null    int64 
 6   INFLOW_START       316 non-null    object
 7   INFLOW_END         316 non-null    object
 8   DATA_SOURCE.1      332 non-null    object
 9   OUTFLOW            678 non-null    int64 
 10  OUTFLOW_START      518 non-null    object
 11  OUTFLOW_END        518 non-null    object
 12  DATA_SOURCE.2      555 non-null    object
 13  ELEVATION          678 non-null    int64 
 14  ELEVATION_START    506 non-null    object
 15  ELEVATION_END      506 non-null    object
 16  DATA_SOURCE.3      553 non-null    object
 1

In [ ]:
time_series_inventory.head()

,DAM_ID,STORAGE,STORAGE_START,STORAGE_END,DATA_SOURCE,INFLOW,INFLOW_START,INFLOW_END,DATA_SOURCE.1,OUTFLOW,...,OUTFLOW_END,DATA_SOURCE.2,ELEVATION,ELEVATION_START,ELEVATION_END,DATA_SOURCE.3,EVAPORATION,EVAPORATION_START,EVAPORATION_END,DATA_SOURCE.4
0,41,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
1,42,1,10/1/70,8/31/20,SCL,1,10/1/70,8/31/20,SCL,1,...,8/31/20,SCL,1,10/1/70,8/31/20,SCL,0,NaN,NaN,NaN
2,54,1,10/1/15,12/31/20,BOR_PN,0,NaN,NaN,NaN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
3,55,1,1/1/65,12/31/20,BOR_PN,1,1/1/65,12/31/20,BOR_PN,1,...,12/31/20,BOR_PN,0,NaN,NaN,NaN,0,NaN,NaN,NaN
4,56,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN,1,...,12/31/20,usgs_12105900,1,1/1/90,12/31/20,ACE_SEA,0,NaN,NaN,NaN


In [ ]:
time_series_inventory_variables

,Variable,Meaning
0,DAM_ID,ID for the dam which can be linked to time ser...
1,STORAGE,Binary to depict if data exists for this varia...
2,STORAGE_START,Earliest date that there is storage data for t...
3,STORAGE_END,Last date that there is storage data for this ...
4,DATA_SOURCE,The data source of this variable. Note that da...
5,INFLOW,Binary to depict if data exists for this varia...
6,INFLOW_START,Earliest date that there is inflow data for th...
7,INFLOW_END,Last date that there is inflow data for this d...
8,OUTFLOW,Earliest date that there is outflow data for t...
9,OUTFLOW_END,Last date that there is outflow data for this ...


In [ ]:
time_series_variables

,Variable,Unit,Unit Text,Time Period,Parameter
0,IN,ms,cubic meters per second,Daily,Reservoir Inflow
1,STOR,MCM,Million Cubic Meters,Daily,Reservoir Storage
2,ELE,m,meters,Daily,Reservoir Elevation
3,OUT,ms,cubic meters per second,Daily,Reservoir Outflow
4,EVAP,MCM,million cubic meters,Daily,Reservoir Evaporation


In [ ]:
reservoir_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 678 entries, 0 to 677
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DAM_ID                 678 non-null    int64  
 1   DAM_NAME               678 non-null    object 
 2   STATE                  678 non-null    object 
 3   AGENCY_CODE            678 non-null    object 
 4   LONG                   678 non-null    float64
 5   LAT                    678 non-null    float64
 6   TIME_SERIES_START      678 non-null    object 
 7   TIME_SERIES_END        677 non-null    object 
 8   INCONSISTENCIES_NOTED  3 non-null      object 
dtypes: float64(2), int64(1), object(6)
memory usage: 47.8+ KB


In [ ]:
reservoir_attributes.head()

,DAM_ID,DAM_NAME,STATE,AGENCY_CODE,LONG,LAT,TIME_SERIES_START,TIME_SERIES_END,INCONSISTENCIES_NOTED
0,41,Ross,Washington,SCL,-121.067305,48.732466,10/1/70,8/31/20,NaN
1,42,Diablo,Washington,SCL,-121.130320,48.714583,10/1/70,8/31/20,NaN
2,54,Wynoochee,Washington,USGS,-123.604561,47.386357,1/1/80,12/31/20,NaN
3,55,Keechelus,Washington,BOR_PN,-121.340171,47.323654,1/1/65,12/31/20,NaN
4,56,Howard A. Hanson Dam,Washington,ACE_SEA,-121.783969,47.277406,1/1/80,12/31/20,NaN


In [ ]:
reservoir_attributes_variables.head()

,Variable,Meaning
0,DAM_ID,Unique DAM_ID for each dam that links reservoi...
1,DAM_NAME,Name of the dam that corresponds to the GRAND_...
2,STATE,State where the dam is located.
3,LONG,The longitude of the dam's location.
4,LAT,The latitude of the dam's location.


In [ ]:
agency_attributes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Agency_Code            47 non-null     object
 1   Agency_Name            47 non-null     object
 2   Data_Disclaimers       8 non-null      object
 3   Inflow/Outflow         44 non-null     object
 4   Additional_Data_Notes  16 non-null     object
 5   Access_ Date           46 non-null     object
 6   Access_Type            46 non-null     object
dtypes: object(7)
memory usage: 2.7+ KB


In [ ]:
agency_attributes.head()

,Agency_Code,Agency_Name,Data_Disclaimers,Inflow/Outflow,Additional_Data_Notes,Access_ Date,Access_Type
0,BOR,Bureau of Reclamation,NaN,Real time,NaN,Spring 2020 - Fall 2020,website download: https://www.usbr.gov/rsvrWat...
1,ACE_LOU,Army Corps of Engineers- Louisville District,Disclaimer: Data may not be 100% accurate,Aggregated spatially and temporally,Given inflow and outflow from multiple sources...,Spring 2020,personal communication
2,ACE_SAC,Army Corps of Engineers-Sacramento District,NaN,NaN,NaN,Spring 2021,website download:https://nicholasinstitute.duk...
3,ACE_OMA,Army Corps of Engineers-Omaha District,NaN,NaN,NaN,Summer 2020,personal communication
4,ACE_STP,Army Corps of Engineers-St. Paul District,All data contained herein is preliminary in na...,NaN,NaN,NaN,NaN


In [ ]:
agency_attributes_variables

,Variable,Meaning
0,Agency_Code,Unique code for each agency which can be trace...
1,Agency_Name,"Agency name and, where applicable, district na..."
2,Data_Disclaimers,Disclaimers on data use provided directly by t...
3,Inflow/Outflow,This columns tells if inflow and/or outflow we...
4,Additional_Data_Notes,Any additional notes for the processing or the...
5,Acces_ Date,Date when the data was last accessed.
6,Access_Type,Personal communication denotes that the data c...


### Selecting inflow data

In [ ]:
# start with California dataset

ca_reservoirs = reservoir_attributes.loc[reservoir_attributes["STATE"] == "California", ["DAM_ID", "DAM_NAME", "AGENCY_CODE", "LAT", "TIME_SERIES_START", "TIME_SERIES_END"]]

In [ ]:
ca_reservoirs[ca_reservoirs["DAM_NAME"] == "Folsom"]

,DAM_ID,DAM_NAME,AGENCY_CODE,LAT,TIME_SERIES_START,TIME_SERIES_END
76,182,Folsom,BOR,38.710426,3/1/55,12/18/18


In [ ]:
folsom_inventory = time_series_inventory[time_series_inventory["DAM_ID"] == 182]

folsom_inventory_outflow_start = folsom_inventory.loc[:, ['DAM_ID', 'INFLOW_START', 'INFLOW_END']]
folsom_inventory_outflow_start

,DAM_ID,INFLOW_START,INFLOW_END
76,182,3/1/55,12/31/20


In [ ]:
"""
INFLOW_START: 3/1/55
INFLOW_END: 12/31/20
"""

folsom_daily_inflow = daily_inflow.loc[:, ['date', '182']]
folsom_daily_inflow['date'] = folsom_daily_inflow['date'].map(lambda d: pd.Timestamp(d))

In [ ]:
# index for 2008-1-1 is 10227
# 182 is the dam ID

folsom_daily_inflow[folsom_daily_inflow["date"] == datetime(2008, 1, 1)]

,date,182
10227,2008-01-01,27.666


In [ ]:
# index for 2018-12-31 is 14262

folsom_daily_inflow[folsom_daily_inflow["date"] == datetime(2018, 12, 31)]

,date,182
14262,2018-12-31,33.895


In [ ]:
# 1/1/2008 - 12/31/18 is the inflow range for the reservoir to use
# Using 20 years worth of data

INFLOW_FROM_2008 = folsom_daily_inflow.iloc[10227:14263]
INFLOW_FROM_2008.reset_index(drop=True, inplace=True)
INFLOW_FROM_2008.rename(columns={"date": "Date", "182": "Inflow"}, inplace=True)
INFLOW_FROM_2008.head()

/var/folders/mn/lp7z86z15611h92hmgbbj2cr0000gn/T/ipykernel_12627/291854448.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  INFLOW_FROM_2008.rename(columns={"date": "Date", "182": "Inflow"}, inplace=True)


,Date,Inflow
0,2008-01-01,27.666
1,2008-01-02,30.044
2,2008-01-03,48.478
3,2008-01-04,90.246
4,2008-01-05,200.200


In [ ]:
INFLOW_FROM_2008.tail()

,Date,Inflow
4031,2018-12-27,58.559
4032,2018-12-28,46.808
4033,2018-12-29,45.250
4034,2018-12-30,42.985
4035,2018-12-31,33.895


### Loading CDEC Water data

#### Near American River, North Fork

In [ ]:
### Auburn Dam Ridge (ADR) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
ADR_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/ADR/ADR_2.xlsx')

# PRECIPITATION, INCREMENTAL, INCHES (daily)
ADR_PRECIP_INCR = pd.read_excel('Reservoir_project/CDEC_Water_Data/ADR/ADR_45.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
ADR_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/ADR/ADR_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
ADR_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/ADR/ADR_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
ADR_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/ADR/ADR_32.xlsx')


In [ ]:
ADR_PRECIP_ACC.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,ADR,D,2,RAIN,NaN,20080101,9.52,NaN,INCHES
1,ADR,D,2,RAIN,NaN,20080102,9.52,NaN,INCHES
2,ADR,D,2,RAIN,NaN,20080103,9.76,NaN,INCHES
3,ADR,D,2,RAIN,NaN,20080104,11.28,NaN,INCHES
4,ADR,D,2,RAIN,NaN,20080105,11.96,NaN,INCHES


In [ ]:
### Huysink (HYS) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
HYS_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_2.xlsx')

# PRECIPITATION, INCREMENTAL, INCHES (daily)
HYS_PRECIP_INCR = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_45.xlsx')

# SNOW DEPTH, INCHES (daily)
HYS_SNOW_DEPTH = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_18.xlsx')

# SNOW, WATER CONTENT, INCHES (daily)
HYS_SNOW_WATER_CONTENT = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_3.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
HYS_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
HYS_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
HYS_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/HYS/HYS_32.xlsx')


In [ ]:
HYS_SNOW_WATER_CONTENT.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,HYS,D,3,SNOW WC,NaN,20080101,7.67,,INCHES
1,HYS,D,3,SNOW WC,NaN,20080102,7.79,,INCHES
2,HYS,D,3,SNOW WC,NaN,20080103,7.79,,INCHES
3,HYS,D,3,SNOW WC,NaN,20080104,8.15,,INCHES
4,HYS,D,3,SNOW WC,NaN,20080105,11.75,,INCHES


#### Near American River, Middle Fork

In [ ]:
### Duncan (DUN) ###

# PRECIPITATION, ACCUMULATED, INCHES (hourly)
DUN_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_2.xlsx')

# WIND, PEAK GUST, MPH (hourly)
DUN_WIND_GUST = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_77.xlsx')

# WIND, SPEED, MPH (hourly)
DUN_WIND_SPEED = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_9.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
DUN_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
DUN_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
DUN_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/DUN/DUN_32.xlsx')


In [ ]:
DUN_WIND_SPEED.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,DUN,H,9,WIND SP,2008-01-01 00:00,2008-01-01 00:00,13.0,,MPH
1,DUN,H,9,WIND SP,2008-01-01 01:00,2008-01-01 01:00,13.0,,MPH
2,DUN,H,9,WIND SP,2008-01-01 02:00,2008-01-01 02:00,15.0,,MPH
3,DUN,H,9,WIND SP,2008-01-01 03:00,2008-01-01 03:00,10.0,,MPH
4,DUN,H,9,WIND SP,2008-01-01 04:00,2008-01-01 04:00,14.0,,MPH


In [ ]:
### Sugar Pine (SGP) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
SGP_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/SGP/SGP_2.xlsx')

# PRECIPITATION, INCREMENTAL, INCHES (daily)
SGP_PRECIP_INCR = pd.read_excel('Reservoir_project/CDEC_Water_Data/SGP/SGP_45.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
SGP_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/SGP/SGP_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
SGP_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/SGP/SGP_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
SGP_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/SGP/SGP_32.xlsx')


In [ ]:
SGP_PRECIP_ACC.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,SGP,D,2,RAIN,NaN,20080101,15.52,,INCHES
1,SGP,D,2,RAIN,NaN,20080102,15.52,,INCHES
2,SGP,D,2,RAIN,NaN,20080103,15.68,,INCHES
3,SGP,D,2,RAIN,NaN,20080104,19.00,,INCHES
4,SGP,D,2,RAIN,NaN,20080105,20.12,,INCHES


#### Near American River, South Fork

In [ ]:
### Forni Ridge (FRN) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
FRN_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_2.xlsx')

# PRECIPITATION, INCREMENTAL, INCHES (daily)
FRN_PRECIP_INCR = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_45.xlsx')

# SNOW DEPTH, INCHES (daily)
FRN_SNOW_DEPTH = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_18.xlsx')

# SNOW, WATER CONTENT, INCHES (daily)
FRN_SNOW_WATER_CONTENT = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_3.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
FRN_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
FRN_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
FRN_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/FRN/FRN_32.xlsx')


In [ ]:
FRN_SNOW_WATER_CONTENT.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,FRN,D,3,SNOW WC,NaN,20080101,7.90,,INCHES
1,FRN,D,3,SNOW WC,NaN,20080102,7.88,,INCHES
2,FRN,D,3,SNOW WC,NaN,20080103,7.86,,INCHES
3,FRN,D,3,SNOW WC,NaN,20080104,8.27,,INCHES
4,FRN,D,3,SNOW WC,NaN,20080105,NaN,,INCHES


In [ ]:
### Pacific House (PFH) ###

# PRECIPITATION, ACCUMULATED, INCHES (daily)
PFH_PRECIP_ACC = pd.read_excel('Reservoir_project/CDEC_Water_Data/PFH/PFH_2.xlsx')

# PRECIPITATION, INCREMENTAL, INCHES (daily)
PFH_PRECIP_INCR = pd.read_excel('Reservoir_project/CDEC_Water_Data/PFH/PFH_45.xlsx')

# TEMPERATURE, AIR AVERAGE, DEG F (daily)
PFH_TEMP_AVG = pd.read_excel('Reservoir_project/CDEC_Water_Data/PFH/PFH_30.xlsx')

# TEMPERATURE, AIR MAXIMUM, DEG F (daily)
PFH_TEMP_MAX = pd.read_excel('Reservoir_project/CDEC_Water_Data/PFH/PFH_31.xlsx')

# TEMPERATURE, AIR MINIMUM, DEG F (daily)
PFH_TEMP_MIN = pd.read_excel('Reservoir_project/CDEC_Water_Data/PFH/PFH_32.xlsx')


In [ ]:
PFH_TEMP_AVG.head()

,STATION_ID,DURATION,SENSOR_NUMBER,SENS_TYPE,DATE TIME,OBS DATE,VALUE,DATA_FLAG,UNITS
0,PFH,D,30,TEMP AV,NaN,20080101,40.0,,DEG F
1,PFH,D,30,TEMP AV,NaN,20080102,39.0,,DEG F
2,PFH,D,30,TEMP AV,NaN,20080103,40.0,,DEG F
3,PFH,D,30,TEMP AV,NaN,20080104,39.0,,DEG F
4,PFH,D,30,TEMP AV,NaN,20080105,33.0,,DEG F


### Cleaning CDEC Water data